In [26]:
import pandas as pd
from bs4 import BeautifulSoup
import pymysql, calendar, time, json
import requests
from threading import Timer
from datetime import datetime
from datetime import timedelta
import re

class DBUpdater:  
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
            password='qor100', db='INVESTAR', charset='utf8')
        
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()
               
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 
     
    def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]                
                    sql = f"REPLACE INTO company_info (code, company, last"\
                        f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                        f"VALUES ({code}, {company}, {today})")
                self.conn.commit()
                print('')              

    def read_naver(self, code, company, pages_to_fetch):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(url,
                headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1] 
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(requests.get(pg_url,
                    headers={'User-agent': 'Mozilla/5.0'}).text)[0])                                          
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
              
            df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
            
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                ' %H:%M'), num+1, company, code, len(df)))

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""  
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])            

    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        pages_to_fetch = 100
        self.update_comp_info()
        try :
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                page_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 100
                config = {'pages_to_fetch':pages_to_fetch}
                json.dump(config, out_file)
                
        self.update_daily_price(pages_to_fetch)
        
        # timer 세팅
        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1, hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext=tmnow.replace(month=tmnow.month+1, day=1, hour=17, minute=0, second=0)
        else :
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0, second=0)
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        t=Timer(secs, self.execute_daily)
        print("Waiting for next update({})... ".format(tmnext.strftime('%Y-%m-%d %H:%M')))
        t.start()
        
if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

class MarketDB :
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
            password='qor100', db='INVESTAR', charset='utf8')
        self.codes = {}
        self.update_comp_info()
                            
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 
        
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
    def get_daily_price(self, code, start_date=None, end_date=None):
        """KRx 종목의 일별 시세를 데이터프레임 형태로 반환
            - code : KRX 종목코드('005930') 또는 상장기업명('삼성전자')
            - start_date : 조회 시작일('2020-01-01'), 미입력 시 1년 오늘
            - end_date : 조회 종료일('2020-12-31'), 미입력 시 오늘 날짜
            """
        if start_date is None:
            one_year_ago = datetime.today() - timedelta(days=365)
            start_date = one_year_ago.strftime('%Y-%m-%d')
            print("start_date is initialized to '{}'".format(start_date))
        else :
            start_lst = re.split('\D+', start_date)
            if (start_lst[0] == ''):
                start_lst = start_lst[1:]
            start_year = int(start_lst[0])
            start_month = int(start_lst[1])
            start_day = int(start_lst[2])
                
            if start_year < 1900 or start_year > 2200:
                print(f'ValueError : start_year({start_year:d}) is wrong.')
                return
            if start_month < 1 or start_month > 12:
                print(f'ValueError : start_month({start_month:d}) is wrong.')
                return
            if start_day < 1 or start_day > 31 :
                print(f'ValueError : start_day({start_day:d}) is wrong.')
                return
            start_date=f"{start_year:04d}-{start_month:02d}-{start_day:02d}"
                
        if (end_date is None):
                end_date = datetime.today().strftime('%Y-%m-%d')
                print("end_date is initialized to '{}'".format(end_date))
        else :
                
            end_lst = re.split('\D+', end_date)
            if (end_lst[0] == ''):
                end_lst = end_lst[1:]
            end_year = int(end_lst[0])
            end_month = int(end_lst[1])
            end_day = int(end_lst[2])
            
            if end_year < 1900 or end_year > 2200:
                print(f'ValueError : end_year({end_year:d}) is wrong.')
                return
            if end_month < 1 or end_month > 12:
                print(f'ValueError : end_month({end_month:d}) is wrong.')
                return
            if end_day < 1 or end_day > 31 :
                print(f'ValueError : end_day({end_day:d}) is wrong.')
                return
            end_date=f"{end_year:04d}-{end_month:02d}-{end_day:02d}"        
        
        codes_keys = list(self.codes.keys())
        codes_values = list(self.codes.values())
        if code in codes_keys:
            pass
        elif code in codes_values:
            idx = codes_values.index(code)
            code = codes_keys[idx]
        else :
            print("ValueError : Code({}) doesn't exist".format(code))

        sql = f"SELECT * FROM daily_price WHERE (code = '{code}' and date >= '{start_date}' and date <= '{end_date}')"
        df = pd.read_sql(sql, self.conn)
        df.index = df['date']
        return df

[2021-06-20 00:35] #0001 REPLACE INTO company_info VALUES (004840, DRB동일, 2021-06-20)
[2021-06-20 00:35] #0002 REPLACE INTO company_info VALUES (155660, DSR, 2021-06-20)
[2021-06-20 00:35] #0003 REPLACE INTO company_info VALUES (001250, GS글로벌, 2021-06-20)
[2021-06-20 00:35] #0004 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 2021-06-20)
[2021-06-20 00:35] #0005 REPLACE INTO company_info VALUES (092220, KEC, 2021-06-20)
[2021-06-20 00:35] #0006 REPLACE INTO company_info VALUES (016380, KG동부제철, 2021-06-20)
[2021-06-20 00:35] #0007 REPLACE INTO company_info VALUES (001390, KG케미칼, 2021-06-20)
[2021-06-20 00:35] #0008 REPLACE INTO company_info VALUES (058860, KTis, 2021-06-20)
[2021-06-20 00:35] #0009 REPLACE INTO company_info VALUES (011070, LG이노텍, 2021-06-20)
[2021-06-20 00:35] #0010 REPLACE INTO company_info VALUES (066570, LG전자, 2021-06-20)
[2021-06-20 00:35] #0011 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2021-06-20)
[2021-06-20 00:35] #0012 REPLACE INTO company_info VAL

[2021-06-20 00:35] #0794 REPLACE INTO company_info VALUES (054630, 에이디칩스, 2021-06-20)
[2021-06-20 00:35] #0795 REPLACE INTO company_info VALUES (298380, 에이비엘바이오, 2021-06-20)
[2021-06-20 00:35] #0796 REPLACE INTO company_info VALUES (003800, 에이스침대, 2021-06-20)
[2021-06-20 00:35] #0797 REPLACE INTO company_info VALUES (357230, 에이치피오, 2021-06-20)
[2021-06-20 00:35] #0798 REPLACE INTO company_info VALUES (224110, 에이텍티앤, 2021-06-20)
[2021-06-20 00:35] #0799 REPLACE INTO company_info VALUES (038110, 에코플라스틱, 2021-06-20)
[2021-06-20 00:35] #0800 REPLACE INTO company_info VALUES (205100, 엑셈, 2021-06-20)
[2021-06-20 00:35] #0801 REPLACE INTO company_info VALUES (067570, 엔브이에이치코리아, 2021-06-20)
[2021-06-20 00:35] #0802 REPLACE INTO company_info VALUES (208860, 엔지스테크널러지, 2021-06-20)
[2021-06-20 00:35] #0803 REPLACE INTO company_info VALUES (183490, 엔지켐생명과학, 2021-06-20)
[2021-06-20 00:35] #0804 REPLACE INTO company_info VALUES (182400, 엔케이맥스, 2021-06-20)
[2021-06-20 00:35] #0805 REPLACE INTO company

[2021-06-20 00:35] #1586 REPLACE INTO company_info VALUES (000100, 유한양행, 2021-06-20)
[2021-06-20 00:35] #1587 REPLACE INTO company_info VALUES (088260, 이리츠코크렙, 2021-06-20)
[2021-06-20 00:35] #1588 REPLACE INTO company_info VALUES (007660, 이수페타시스, 2021-06-20)
[2021-06-20 00:35] #1589 REPLACE INTO company_info VALUES (093230, 이아이디, 2021-06-20)
[2021-06-20 00:35] #1590 REPLACE INTO company_info VALUES (014990, 인디에프, 2021-06-20)
[2021-06-20 00:35] #1591 REPLACE INTO company_info VALUES (015860, 일진홀딩스, 2021-06-20)
[2021-06-20 00:35] #1592 REPLACE INTO company_info VALUES (317400, 자이에스앤디, 2021-06-20)
[2021-06-20 00:35] #1593 REPLACE INTO company_info VALUES (000950, 전방, 2021-06-20)
[2021-06-20 00:35] #1594 REPLACE INTO company_info VALUES (348950, 제이알글로벌리츠, 2021-06-20)
[2021-06-20 00:35] #1595 REPLACE INTO company_info VALUES (036420, 제이콘텐트리, 2021-06-20)
[2021-06-20 00:35] #1596 REPLACE INTO company_info VALUES (030000, 제일기획, 2021-06-20)
[2021-06-20 00:35] #1597 REPLACE INTO company_info VAL

[2021-06-20 00:35] #2241 REPLACE INTO company_info VALUES (078590, OQP, 2021-06-20)
[2021-06-20 00:35] #2242 REPLACE INTO company_info VALUES (178920, PI첨단소재, 2021-06-20)
[2021-06-20 00:35] #2243 REPLACE INTO company_info VALUES (091340, S&K폴리텍, 2021-06-20)
[2021-06-20 00:35] #2244 REPLACE INTO company_info VALUES (036120, SCI평가정보, 2021-06-20)
[2021-06-20 00:35] #2245 REPLACE INTO company_info VALUES (036540, SFA반도체, 2021-06-20)
[2021-06-20 00:35] #2246 REPLACE INTO company_info VALUES (049470, SGA, 2021-06-20)
[2021-06-20 00:35] #2247 REPLACE INTO company_info VALUES (016250, SGC이테크건설, 2021-06-20)
[2021-06-20 00:35] #2248 REPLACE INTO company_info VALUES (089230, THE E&M, 2021-06-20)
[2021-06-20 00:35] #2249 REPLACE INTO company_info VALUES (161570, THE MIDONG, 2021-06-20)
[2021-06-20 00:35] #2250 REPLACE INTO company_info VALUES (032540, TJ미디어, 2021-06-20)
[2021-06-20 00:35] #2251 REPLACE INTO company_info VALUES (038340, UCI, 2021-06-20)
[2021-06-20 00:35] #2252 REPLACE INTO company

[2021-06-20 00:35] #0001 동화약품 (000020) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:36] #0002 KR모터스 (000040) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:36] #0003 경방 (000050) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:36] #0004 메리츠화재 (000060) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:36] #0005 삼양홀딩스 (000070) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:37] #0006 하이트진로 (000080) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:37] #0007 유한양행 (000100) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:37] #0008 CJ대한통운 (000120) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:37] #0009 하이트진로홀딩스 (000140) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:37] #0010 두산 (000150) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:37] #0011 성창기업지주 (000180) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 00:38] #0012 DL (000210) : 1000 rows > REPLACE INTO daily_price [OK

...

[2021-06-20 07:38] #1973 패션플랫폼 (225590) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:38] #1974 미애부 (225850) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:38] #1975 엠앤씨생명과학 (225860) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:39] #1976 잇츠한불 (226320) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:39] #1977 신테카바이오 (226330) : 372 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:39] #1978 본느 (226340) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:39] #1979 KH E&T (226360) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:39] #1980 오스테오닉 (226400) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:39] #1981 한송네오텍 (226440) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:40] #1982 올릭스 (226950) : 720 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:40] #1983 디자인 (227100) : 640 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:40] #1984 도부마스크 (227420) : 1000 rows > REPLACE INTO daily_price [OK]
[

[2021-06-20 07:55] #2072 네오셈 (253590) : 790 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:55] #2073 루트락 (253610) : 119 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:55] #2074 수젠텍 (253840) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:56] #2075 자비스 (254120) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:56] #2076 SG (255220) : 835 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:56] #2077 야스 (255440) : 911 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:56] #2078 한독크린텍 (256150) : 441 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:56] #2079 포인트엔지니어링 (256630) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:56] #2080 한국비엔씨 (256840) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:57] #2081 케이피에스 (256940) : 1000 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:57] #2082 명성티엔에스 (257370) : 680 rows > REPLACE INTO daily_price [OK]
[2021-06-20 07:57] #2083 나우코스 (257990) : 116 rows > REPLACE INTO daily_price [OK]
[2021-06-2

[2021-06-20 08:12] #2171 SK케미칼 (285130) : 850 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:12] #2172 노바텍 (285490) : 649 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:12] #2173 라이프사이언스테크놀로지 (285770) : 858 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:12] #2174 씨엔티드림 (286000) : 128 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:12] #2175 나노브릭 (286750) : 454 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:12] #2176 롯데정보통신 (286940) : 713 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:12] #2177 제이시스메디칼 (287410) : 768 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:12] #2178 브릿지바이오 (288330) : 369 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:13] #2179 나라소프트 (288490) : 634 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:13] #2180 에스퓨얼셀 (288620) : 663 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:13] #2181 아이스크림에듀 (289010) : 480 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:13] #2182 SV인베스트먼트 (289080) : 728 rows > REPLACE INTO daily_pr

[2021-06-20 08:23] #2271 TS트릴리온 (317240) : 530 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2272 한화에스비아이스팩 (317320) : 527 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2273 덕산테코피아 (317330) : 464 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2274 자이에스앤디 (317400) : 401 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2275 캐리소프트 (317530) : 407 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2276 퀀타매트릭스 (317690) : 130 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2277 슈프리마아이디 (317770) : 465 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2278 에스피시스템스 (317830) : 456 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2279 대모 (317850) : 471 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:23] #2280 노드메이슨 (317860) : 379 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:24] #2281 엔바이오니아 (317870) : 410 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:24] #2282 한국바이오젠 (318000) : 465 rows > REPLACE INTO daily_pric

[2021-06-20 08:29] #2369 데이드림엔터 (348840) : 207 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:29] #2370 제이알글로벌리츠 (348950) : 213 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:29] #2371 이베스트스팩5호 (349720) : 270 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:29] #2372 이지스레지던스리츠 (350520) : 215 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:29] #2373 IBKS제14호스팩 (351320) : 247 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:29] #2374 이삭엔지니어링 (351330) : 41 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:29] #2375 IBKS제13호스팩 (351340) : 230 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:29] #2376 씨앤씨인터내셔널 (352480) : 24 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:30] #2377 씨앤투스성진 (352700) : 97 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:30] #2378 클리노믹스 (352770) : 133 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:30] #2379 하이브 (352820) : 169 rows > REPLACE INTO daily_price [OK]
[2021-06-20 08:30] #2380 인바이오 (352940) : 128 rows > REPLACE INTO 